In [23]:
import os
import re
import time
import jieba
import numpy as np
import networkx as nx
import time

from collections import defaultdict
from spiders import PcbSpider
from pagerank import PRIterator
from pagerank import PRMapReduce
from operator import and_
from functools import reduce
from scipy.spatial.distance import cosine
from sklearn.feature_extraction.text import TfidfVectorizer

# 获取页面内容

In [6]:
def cut(string): 
    return ' '.join(jieba.cut(string))

def text_clean(text):
    return cut(' '.join(re.split('[^\w]',text)))

## 爬取数据并存入文件

In [15]:
pcbspider = PcbSpider()
theme_urls = pcbspider.get_theme_urls()
content = defaultdict()
start_time = time.time()
n = 1

file_path = os.path.join('../data/', 'PcbSpiderData_index.txt')
with open(file_path, 'w', encoding='gbk') as fi:
    for theme_url in theme_urls:
        post_urls = pcbspider.get_more_post_urls(theme_url)
        for post_url in post_urls:
            try:
                post_contents = pcbspider.get_more_post_content(post_url)
                content[post_url] = [text_clean(post_contents[0]), post_contents[1]]
                fi.writelines(content)
            except:
                print('error: {}'.format(post_url))

        if (time.time() - start_time) // 180 > n:
            print('has been spend {} minutes'.format(
                n*3))
            n += 1

error: http://www.pcbbbs.com/forum.php?mod=viewthread&tid=279935&extra=page%3D4
error: http://www.pcbbbs.com/forum.php?mod=viewthread&tid=246905&extra=page%3D1
error: http://www.pcbbbs.com/forum.php?mod=viewthread&tid=276055&extra=page%3D1
has been spend 3m
has been spend 6m
has been spend 9m
error: http://www.pcbbbs.com/forum.php?mod=viewthread&tid=228994&extra=page%3D2
error: http://www.pcbbbs.com/forum.php?mod=viewthread&tid=220579&extra=page%3D2
has been spend 12m
has been spend 15m
error: http://www.pcbbbs.com/forum.php?mod=viewthread&tid=244305&extra=page%3D3
has been spend 18m
has been spend 21m
has been spend 24m
error: http://www.pcbbbs.com/forum.php?mod=viewthread&tid=266551&extra=page%3D3
error: http://www.pcbbbs.com/forum.php?mod=viewthread&tid=258844&extra=page%3D5
has been spend 27m
error: http://www.pcbbbs.com/forum.php?mod=viewthread&tid=247588&extra=page%3D3
has been spend 30m
has been spend 33m
has been spend 36m
has been spend 39m
has been spend 42m
has been spend 45

## 加载文件

In [8]:
file_path_index = os.path.join('../data/', 'PcbSpiderData_index.txt')
with open(file_path_index, encoding='gbk') as fi:
    spiderdata_index = eval(fi.readlines()[0])

# spiderdata_index结构：key为当前网页链接，value为[str,list]，str为文本内容，list为跳转链接

In [24]:
corpus = []
# 将链接与位置记录下来，可以通过索引快速找到链接
link_idx = defaultdict()
idx = 0
for key in spiderdata_index:
    link_idx[idx] = key
    corpus.append(cut(spiderdata_index[key][0]))
    idx += 1

# 训练TFIDF模型

In [10]:
# 训练tfidf模型
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(corpus)
# 转置，使词向量变成索引
transposed_tfidf = tfidf.transpose() 
transposed_tfidf_array = transposed_tfidf.toarray()

In [11]:
def get_word_id(word):
    return vectorizer.vocabulary_.get(word)

def get_candidates_ids(string):
    return [get_word_id(word) for word in cut(string).split()]

def document_cols(word_idx):
    return np.where(transposed_tfidf_array[word_idx])[0]

def and_operation(arrays):
    return reduce(and_, arrays)

def keyword_bold(query, input_str):
    pattern = re.compile('({})'.format('|'.join(cut(query).split())))
    return pattern.sub(repl='** \g<1> ** ',string=input_str)

def search_enginer(query):
#     求取查询语句分词后在模型中对应的文档id
    candidates_ids = get_candidates_ids(query)
    document = [set(document_cols(candidate))for candidate in candidates_ids]
    retrieval_documents = and_operation(document)
    
#     查询的语句在模型中转化为向量，根据查询语句和内容相似程度倒序排列
    query_vector = vectorizer.transform([cut(query)]).toarray()[0]
    vector_with_ids = [(tfidf[doc_idx], doc_idx) for doc_idx in retrieval_documents]
    sorted_vector_with_ids = sorted(vector_with_ids, key=lambda x:cosine(x[0].toarray(),query_vector), reverse=True)
    
    for vector_with_id in sorted_vector_with_ids:
        output = keyword_bold(query, corpus[vector_with_id[1]])
        yield ''.join(output.split())

In [12]:
with open('../data/output/sz_result.md','w') as fi:
    for i, document in enumerate(search_enginer('正确理解 器件镀金厚度单位')):
        fi.write('##### search result {}\n'.format(i))
        fi.write(document+'\n')
    print('done!')

done!


In [15]:
with open('../data/output/sz_result.md') as fi:
    for i in range(10):
        print(fi.readline())

##### search result 0

查看11538回复14置顶精华贴部分汇总按发布时间排序_20130922复制链接wanghanqwanghanq当前离线积分1713178兑换点点声望度金钱元银行存款元贡献度精华电梯直达1发表于2013922102354只看该作者倒序浏览阅读模式pcbmarginright0本帖最后由wanghanq于2018422326编辑备注此类汇总可能的弊病论坛改版导致主题帖链接地址失效故当链接地址失效时可通过站内搜索引擎重新定位地址置顶1来是希望更多的网友看到并参与话题2来是对楼主表示鼓励其他主题帖中的广告信息可能会误导网友误以为是论坛推广行为参与的网友尽可能避免在主题帖中发布广告信息非论坛自身的广告信息再如百度那些用钱来衡量排名顺序的信息建议网友酌情考虑是否选择并信任欢迎网友发布原创文章只是不希望您发布原创文章的主要目的是为发布推广信息有前劲无后劲哆啦梦发表于201024080500含个人信息酌情考虑2010新年献礼2009Layout鲫鱼塘宝典全集httpwwwpcbbbscomforumphpmodviewthreadtid217048fromuid63313此帖中下载地址已经失效相关内容下载当前可参关联帖内的下载地址关联贴2012元旦献礼Layout鲫鱼塘宝典2012元旦篇httpwwwpcbbbscomforumphpmodviewthreadtid243350fromuid63313了解些背景我不是高手我只是热衷于分享layout鲫鱼塘201203011650很长一段时间因为网友看到我发的原创贴子以为我是高手其实是不是高手自己心里知道我平时工作中找到一些方法软件使用经验喜欢写下来以防忘记了感觉可能对同行有所帮助也就发到网上去了可能这些方法不会用很多次但是有时用一次也可能省不少时间我为什么要把这些资料分享出来呢可能有人说就是臭显摆呗这样说其实是高在抬我因为就这些资料有什么好显摆的在如今金钱至少的社会有钱才能显摆别的就不用说什么了虽然我不是想显摆什么但是我想把自己的经验和方法分享出来总不能说有错吧人类社会之所以发展得这么快不就是我们学习了先人的经验吗当然我的经验无法跟先人的智慧相比但我也想尽自己的绵薄之力人活一世总要为这个世界做点什么这并不是什么高尚的想法是社会的进步和发展需要我们这样做虽然我个人的力量很微弱但不是有那么

# PageRank

In [16]:
# 基于爬取到的链接，与每个链接可以转跳的链接构建图
pcburl_graph = nx.DiGraph()

In [17]:
pcburl_graph.add_nodes_from(spiderdata_index.keys())

In [18]:
for url in spiderdata_index:
    if len(spiderdata_index[url][1]) != 0:
        for url2 in spiderdata_index[url][1]:
            pcburl_graph.add_edge(url,url2)

## 常规的PageRank

In [150]:
# 由于nx自带的pagerank训练时内存超出限制，因此用自己写的PageRank进行计算
pr = PRIterator(pcburl_graph)
start_time = time.time()
pcb_pagerank = pr.page_rank()
end_time = time.time()
print('has been append {} s'.format(end_time-start_time))

This is NO.1 iteration
This is NO.2 iteration
This is NO.3 iteration
This is NO.4 iteration
This is NO.5 iteration
This is NO.6 iteration
This is NO.7 iteration
This is NO.8 iteration
This is NO.9 iteration
This is NO.10 iteration
This is NO.11 iteration
This is NO.12 iteration
This is NO.13 iteration
This is NO.14 iteration
This is NO.15 iteration
This is NO.16 iteration
This is NO.17 iteration
finished in http://www.pcbbbs.com/viewthread.php?tid=212392  iterations!
has been append 2774.948380470276 s


## 基于MapReduce的PageRank

In [19]:
mr_pr = PRMapReduce(pcburl_graph,min_delta=1e-6)

In [21]:
start_time = time.time()
pcb_pagerank = mr_pr.page_rank()
end_time = time.time()
print('has been append {} s'.format(end_time-start_time))

Change: 0.008841148770971818
Change: 0.0003947569386056981
Change: 0.00019245892818916253
Change: 9.815405337647277e-05
Change: 5.005856722200121e-05
Change: 2.5529869283220648e-05
Change: 1.302023333444246e-05
Change: 6.640319000565469e-06
Change: 3.38656269028854e-06
Change: 1.7271469720471955e-06
Change: 8.808449557440767e-07
finished in 10 iterations
has been append 0.3732156753540039 s


In [25]:
link_idx[1]

'http://www.pcbbbs.com/forum.php?mod=viewthread&tid=289116&extra=page%3D1'

In [27]:
pcb_pagerank['http://www.pcbbbs.com/forum.php?mod=viewthread&tid=289116&extra=page%3D1'][0]

0.00013114490998874243

# 通过PageRank对输出网页排序

In [71]:
# 结合两列的排序情况，如同一组数据排序分别为1、3，则最终分值为4
def combine_sorted(input_list, key1, key2):

    sorted1 = sorted(input_list, key=key1, reverse=True)
    sorted2 = sorted(input_list, key=key2, reverse=True)
    for value in input_list:
        print(sorted1.index(value) + sorted2.index(value))
        value += [sorted1.index(value) + sorted2.index(value)]
    return sorted(input_list, key=lambda x: x[-1])

In [78]:
def new_search_enginer(query):
    #     求取查询语句分词后在模型中对应的文档id
    candidates_ids = get_candidates_ids(query)
    document = [set(document_cols(candidate)) for candidate in candidates_ids]
    retrieval_documents = and_operation(document)
    #     查询的语句在模型中转化为向量，根据查询语句和内容相似程度倒序排列
    query_vector = vectorizer.transform([cut(query)]).toarray()[0]
    #     link_idx[doc_idx]为链接，pcb_pagerank[link_idx[doc_idx]][0]为pagerank值
    vector_with_ids = [(tfidf[doc_idx], doc_idx,
                        pcb_pagerank[link_idx[doc_idx]][0])
                       for doc_idx in retrieval_documents]

    #     根据相似度进行排序
    sorted_vector_with_ids = combine_sorted(
        vector_with_ids,
        key1=lambda x: cosine(x[0].toarray(), query_vector),
        key2=lambda x: x[2])

    for vector_with_id in sorted_vector_with_ids:
        output = keyword_bold(query, corpus[vector_with_id[1]])
        yield ''.join(output.split())

In [79]:
with open('../data/output/sz_result2.md','w') as fi:
    for i, document in enumerate(search_enginer('正确理解 器件镀金厚度单位')):
        fi.write('##### search result {}\n'.format(i))
        fi.write(document+'\n')
    print('done!')

done!


In [80]:
with open('../data/output/sz_result2.md') as fi:
    for i in range(10):
        print(fi.readline())

##### search result 0

查看11538回复14置顶精华贴部分汇总按发布时间排序_20130922复制链接wanghanqwanghanq当前离线积分1713178兑换点点声望度金钱元银行存款元贡献度精华电梯直达1发表于2013922102354只看该作者倒序浏览阅读模式pcbmarginright0本帖最后由wanghanq于2018422326编辑备注此类汇总可能的弊病论坛改版导致主题帖链接地址失效故当链接地址失效时可通过站内搜索引擎重新定位地址置顶1来是希望更多的网友看到并参与话题2来是对楼主表示鼓励其他主题帖中的广告信息可能会误导网友误以为是论坛推广行为参与的网友尽可能避免在主题帖中发布广告信息非论坛自身的广告信息再如百度那些用钱来衡量排名顺序的信息建议网友酌情考虑是否选择并信任欢迎网友发布原创文章只是不希望您发布原创文章的主要目的是为发布推广信息有前劲无后劲哆啦梦发表于201024080500含个人信息酌情考虑2010新年献礼2009Layout鲫鱼塘宝典全集httpwwwpcbbbscomforumphpmodviewthreadtid217048fromuid63313此帖中下载地址已经失效相关内容下载当前可参关联帖内的下载地址关联贴2012元旦献礼Layout鲫鱼塘宝典2012元旦篇httpwwwpcbbbscomforumphpmodviewthreadtid243350fromuid63313了解些背景我不是高手我只是热衷于分享layout鲫鱼塘201203011650很长一段时间因为网友看到我发的原创贴子以为我是高手其实是不是高手自己心里知道我平时工作中找到一些方法软件使用经验喜欢写下来以防忘记了感觉可能对同行有所帮助也就发到网上去了可能这些方法不会用很多次但是有时用一次也可能省不少时间我为什么要把这些资料分享出来呢可能有人说就是臭显摆呗这样说其实是高在抬我因为就这些资料有什么好显摆的在如今金钱至少的社会有钱才能显摆别的就不用说什么了虽然我不是想显摆什么但是我想把自己的经验和方法分享出来总不能说有错吧人类社会之所以发展得这么快不就是我们学习了先人的经验吗当然我的经验无法跟先人的智慧相比但我也想尽自己的绵薄之力人活一世总要为这个世界做点什么这并不是什么高尚的想法是社会的进步和发展需要我们这样做虽然我个人的力量很微弱但不是有那么